# Preparing Data

In [15]:
import glob
import re
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pywt import wavedec
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix,mean_squared_error,precision_score,recall_score,f1_score
import seaborn as sns
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import classification_report , roc_curve, f1_score, accuracy_score, recall_score , roc_auc_score,make_scorer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier ,  GradientBoostingClassifier 
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from scipy import signal
from scipy.signal import butter, filtfilt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV



In [2]:
# Dataset Classes:
# ________________

# Yukari-->Up
# Asagi-->Down
# Sag-->Right 
# Sol-->Left
# Kirp-->Blink 

In [3]:
# Define the class labels
class_labels = {
    'yukari': 0,
    'yukarı':0 ,
    'yukarih': 0,
    'yukarıh':0 ,
    'yukari': 0,
    'asagi': 1,
    'sag': 2,
    'sol': 3,
    'kirp': 4
}

In [4]:
class_names = {
    0: 'Up',
    1: 'Down',
    2: 'Right',
    3: 'Left',
    4: 'Blink'
}

In [5]:
 h=[]
classes=class_labels.keys()

In [6]:
classes

dict_keys(['yukari', 'yukarı', 'yukarih', 'yukarıh', 'asagi', 'sag', 'sol', 'kirp'])

In [7]:
 with open('data handling.csv', 'a') as csv_file:
        for filename in glob.glob('3-class/*.txt'):
            # Get the class label from the filename

            print(filename)
            h = filename.split('\\')[-1].split('.')[0]
            matches = re.split(r'(\d+)', h)
            
            if matches[0] in classes:
                with open(filename, 'r') as file:
                    # Read the contents of the file and split it into lines
                    lines = file.read()
                lines=lines.replace('\n',',')
                for line in lines:
                    csv_file.write(line)
                label = class_labels[matches[0]]
                csv_file.write(str(label)+'\n')

3-class\asagi10h.txt
3-class\asagi10v.txt
3-class\asagi11h.txt
3-class\asagi11v.txt
3-class\asagi12h.txt
3-class\asagi12v.txt
3-class\asagi13h.txt
3-class\asagi13v.txt
3-class\asagi14h.txt
3-class\asagi14v.txt
3-class\asagi15h.txt
3-class\asagi15v.txt
3-class\asagi16h.txt
3-class\asagi16v.txt
3-class\asagi17h.txt
3-class\asagi17v.txt
3-class\asagi18h.txt
3-class\asagi18v.txt
3-class\asagi19h.txt
3-class\asagi19v.txt
3-class\asagi1h.txt
3-class\asagi1v.txt
3-class\asagi20h.txt
3-class\asagi20v.txt
3-class\asagi2h.txt
3-class\asagi2v.txt
3-class\asagi3h.txt
3-class\asagi3v.txt
3-class\asagi4h.txt
3-class\asagi4v.txt
3-class\asagi5h.txt
3-class\asagi5v.txt
3-class\asagi6h.txt
3-class\asagi6v.txt
3-class\asagi7h.txt
3-class\asagi7v.txt
3-class\asagi8h.txt
3-class\asagi8v.txt
3-class\asagi9h.txt
3-class\asagi9v.txt
3-class\Edit2.txt
3-class\Edit2h.txt
3-class\Edit2v.txt
3-class\kayit.txt
3-class\kirp10h.txt
3-class\kirp10v.txt
3-class\kirp111h.txt
3-class\kirp11h.txt
3-class\kirp11v.txt
3-c

# Load data

In [2]:
names=np.arange(0,252)

In [3]:
names

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [4]:
data =pd.read_csv('data handling.csv',names=names)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,123,126,134,131,125,131,130,127,133,128,...,143,139,143,141,139,138,142,136,137,1
1,123,122,127,130,123,123,127,128,125,125,...,134,133,130,136,135,130,139,136,129,1
2,127,135,130,135,129,127,133,131,130,132,...,143,140,139,144,142,139,139,143,134,1
3,120,132,120,127,126,125,120,125,122,118,...,128,137,131,124,133,129,124,131,130,1
4,133,137,134,132,137,136,135,136,136,130,...,143,141,148,148,144,149,147,150,150,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Columns: 252 entries, 0 to 251
dtypes: int64(252)
memory usage: 439.2 KB


# Extract features and labels

In [7]:
labels = data.iloc[:, -1]
x = data.iloc[:, :-1]

# standardize the data using z-score normalization



In [8]:
x = (x - np.mean(x)) / np.std(x)

# apply wavelet transform to the data

In [9]:
def wavelet_transform(row):
    coeffs = pywt.wavedec(row, 'db1', level=3)
    return np.concatenate(coeffs)

In [10]:
x = x.apply(wavelet_transform, axis=1)

# extract statistical features from the wavelet coefficients

In [11]:
features = pd.DataFrame(x.apply(lambda x: [np.mean(x), np.std(x), np.median(x), np.min(x), np.max(x)]).tolist(),
                        columns=['mean', 'std', 'median', 'min', 'max'])

# split the data into training and testing sets

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.1, random_state=42)

# TRAINING

In [16]:
# train Random Forest classifier
rf = RandomForestClassifier(n_estimators=400, max_depth=11, random_state=42)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=11, n_estimators=400, random_state=42)

In [17]:
# train Gradient Boosting classifier
gb = GradientBoostingClassifier(n_estimators=520, learning_rate=0.11, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.11, n_estimators=520,
                           random_state=42)

In [18]:
# train K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [19]:
# train Logistic Regression classifier
lr = LogisticRegression()
lr.fit(X_train, y_train)

C:\Users\AHMED\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
# train Support Vector Machine classifier
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X_train, y_train)

SVC(random_state=42)

# evaluate the classifiers on the testing set

In [21]:
models = {'Random Forest': rf, 'Gradient Boosting': gb, 'K-NN': knn, 'Logistic Regression': lr, 'SVM': svm}
for name, model in models.items():
    y_pred = model.predict(X_test)
    train_accuracy=model.score(X_train,y_train)
    print( 'train accuracy: ',train_accuracy)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("{} Classifier - Accuracy: {:.2f}, F1-score: {:.2f}".format(name, accuracy, f1))

NameError: name 'mlp' is not defined